# 1. Parameters & Libraries

In [ ]:
n_fold=1
step=1
rand_seeds=n_fold*step

rand=100
save_model_name = "./unet_best1.model"
save_end_name = "./unet_end1.model"
transfar_model_01 = "../input/binary-class-fo01-demo-181109/unet_best1.model"
fold_path = '../input/simple-unet-100-5-fold-df/fold_df.csv'

start_lr = 1e-4
epochs = 1
batch_size = 32
dropout_rate = 0.3
weight = 1.0

In [ ]:
import os
import sys
import random

import pandas as pd
import numpy as np
import six
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Concatenate,UpSampling2D,Input,Dropout,BatchNormalization,Activation,Add,GlobalAveragePooling2D,Dense,Multiply
from keras.layers.core import Lambda, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers
from keras.regularizers import l2

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img

import gc
gc.collect()

In [ ]:
import time
from datetime import datetime, timedelta, timezone
from contextlib import contextmanager
JST = timezone(timedelta(hours=+9), 'JST')
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} -> done in {:.0f}s".format(title, time.time() - t0))
    print("Executed time -> {}\n".format(datetime.now(JST)))

with timer('end'):
    print('finish!')

In [ ]:
# Set some parameters
im_width = 101
im_height = 101
im_chan = 1
basicpath = '../input/tgs-salt-identification-challenge/'
path_train = basicpath + 'train/'
path_test = basicpath + 'test/'

path_train_images = path_train + 'images/'
path_train_masks = path_train + 'masks/'
path_test_images = path_test + 'images/'

# 2. Helpers & Data engineering

In [ ]:
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return np.pad(img, [(img_size_target-img_size_ori)//2,(img_size_target-img_size_ori)-(img_size_target-img_size_ori)//2], 'edge')
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return img[(img_size_target-img_size_ori)//2:img_size_ori+(img_size_target-img_size_ori)//2, (img_size_target-img_size_ori)//2:img_size_ori+(img_size_target-img_size_ori)//2]

In [ ]:
# Loading of training/testing ids and depths

train_df = pd.read_csv(basicpath+"train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv(basicpath+"depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)

In [ ]:
train_df["images"] = [np.array(load_img(path_train_images+"{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img(path_train_masks+"{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fold_df = pd.read_csv(fold_path)

In [ ]:
ids_valid = fold_df[fold_df['n_fold']==n_fold].valid_idx.values
ids_train = np.array([i for i in np.arange(train_df.shape[0]) if i not in ids_valid])

In [ ]:
print("train shape: {}".format(ids_train.shape))
print("valid shape: {}".format(ids_valid.shape))

In [ ]:
x_train = np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_train]
x_valid = np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_valid]
y_train = np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_train]
y_valid = np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_valid]

In [ ]:
x_train_org = x_train
x_valid_org = x_valid
y_train_org = y_train
y_valid_org = y_valid

In [ ]:
y_train = np.any(y_train==1, axis=(1,2,3))*1
y_valid = np.any(y_valid==1, axis=(1,2,3))*1

# 3. Build a model

In [ ]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [ ]:
# cSE
def channel_gate(blockInput, num_filters):
    x = GlobalAveragePooling2D()(blockInput)
    x = Dense(num_filters//2, activation='relu')(x)
    x = Dense(num_filters, activation='sigmoid')(x)
    return Multiply()([blockInput, x])

# sSE
def spatial_gate(blockInput):
    x = Conv2D(1, (1, 1), strides=(1,1), padding="same", activation='sigmoid')(blockInput)
    return Multiply()([blockInput, x])

# SE
def se_block(blockInput, num_filters):
    return Add()([channel_gate(blockInput, num_filters), spatial_gate(blockInput)])

In [ ]:
# Build model
def build_model(input_layer, start_neurons = 16, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    FC = (Flatten())(convm)
    outputs = (Dense(4096))(FC) 
    outputs = (Activation('relu'))(outputs)
    outputs = (Dropout(DropoutRatio/2))(outputs)
    outputs = (Dense(256))(outputs)
    outputs = (Activation('relu'))(outputs)
    outputs = (Dropout(DropoutRatio/4))(outputs)
    outputs = (Dense(16))(outputs) 
    outputs = (Activation('relu'))(outputs)
    outputs = (Dropout(DropoutRatio/4))(outputs)
    outputs = (Dense(1))(outputs)
    outputs = (Activation('sigmoid'))(outputs)
    
    return outputs

# 4. Train my model

## 4.1. Augmentation

In [ ]:
def do_brightness_multiply(image, alpha=1):
    image = alpha*image
    image = np.clip(image, 0, 1)
    return image

def do_brightness_aug(x_train, alpha):
    tmp = []
    for i in np.arange(x_train[:, :, :, 0].shape[0]):
        x_train_bright = do_brightness_multiply(x_train[i, :, :, 0], alpha=(100+alpha)/100)
        tmp.append(x_train_bright.reshape(1, img_size_target, img_size_target, 1))

    x_train_aug = np.concatenate(tmp, axis=0)
    del tmp, x_train_bright; gc.collect()
    return x_train_aug

In [ ]:
def do_brightness_multiply2(image, mask, alpha=1):
    image = alpha*image
    image = np.clip(image, 0, 1)
    return image, mask

In [ ]:
def do_flip_transpose2(image, mask, type=0):
    #choose one of the 8 cases
    if type==1: #rotate90
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    if type==2: #rotate180
        image = cv2.flip(image,-1)
    if type==3: #rotate270
        image = image.transpose(1,0)
        image = cv2.flip(image,0)
    if type==4: #flip left-right
        image = cv2.flip(image,1)
    if type==5: #flip up-down
        image = cv2.flip(image,0)
    if type==6:
        image = cv2.flip(image,1)
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    if type==7:
        image = cv2.flip(image,0)
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    return image, mask

def do_flip_transpose(image, type=0):
    #choose one of the 8 cases
    if type==1: #rotate90
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    if type==2: #rotate180
        image = cv2.flip(image,-1)
    if type==3: #rotate270
        image = image.transpose(1,0)
        image = cv2.flip(image,0)
    if type==4: #flip left-right
        image = cv2.flip(image,1)
    if type==5: #flip up-down
        image = cv2.flip(image,0)
    if type==6:
        image = cv2.flip(image,1)
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    if type==7:
        image = cv2.flip(image,0)
        image = image.transpose(1,0)
        image = cv2.flip(image,1)
    return image

In [ ]:
#Data augmentation
def do_flip_transpose_aug2(x_train, y_train, flip_type):
    tmp_x = []
    tmp_y = []
    for i in np.arange(x_train[:, :, :, 0].shape[0]):
        x_train_flip, y_train_flip = do_flip_transpose2(x_train[i, :, :, 0], y_train[i], type=flip_type)
        tmp_x.append(x_train_flip.reshape(1, img_size_target, img_size_target, 1))
        tmp_y.append(y_train_flip)
        
    x_train_aug = np.concatenate(tmp_x, axis=0)
    y_train_aug = np.array(tmp_y)
    del tmp_x, tmp_y, x_train_flip, y_train_flip; gc.collect()
    return x_train_aug, y_train_aug

def do_flip_transpose_aug(x_train, flip_type):
    tmp_x = []
    for i in np.arange(x_train[:, :, :, 0].shape[0]):
        x_train_flip = do_flip_transpose(x_train[i, :, :, 0], type=flip_type)
        tmp_x.append(x_train_flip.reshape(1, img_size_target, img_size_target, 1))
        
    x_train_aug = np.concatenate(tmp_x, axis=0)
    del tmp_x, x_train_flip; gc.collect()
    return x_train_aug

In [ ]:
def random_aug(image, mask):
    np.random.seed()
    i = np.random.randint(0, 8)
    if i in (4, 5, 6, 7):
        image, mask = do_flip_transpose2(image, mask, 4)
    if i in (2, 6):
        image, mask = do_brightness_multiply2(image, mask, 1.08)
    if i in (3, 7):
        image, mask = do_brightness_multiply2(image, mask, 0.92)
    return image, mask

In [ ]:
from numpy.random import*
def batch_generator(x, y, w=1.0, batch_size=32):
    '''
    Return a random image from x, y
    '''
    while True:
        rand_index = np.arange(x.shape[0])
        np.random.seed()
        shuffle(rand_index)
        current_index = 0
        while current_index + batch_size <= x.shape[0]:
            batch_index = rand_index[current_index:current_index + batch_size]
            tmp_x = []
            tmp_y = []
            tmp_w = []
            for i in batch_index:
                image, mask = random_aug(x[i], y[i])
                if (mask>0.5).sum() < 8:
                    weight = w
                else:
                    weight = 1.0
                tmp_x.append(image.reshape(1, img_size_target, img_size_target, 1))
                tmp_y.append(mask)
                tmp_w.append(weight)
            image = np.concatenate(tmp_x, axis=0)
            mask = np.array(tmp_y)
            weight = np.array(tmp_w)
            current_index += batch_size
            yield image, mask, weight

## 4.2. Model compilation

In [ ]:
input_layer = Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16, dropout_rate)
model = Model(input_layer, output_layer)
c = optimizers.adam(lr = start_lr)
model.compile(loss='binary_crossentropy', optimizer=c, metrics=['accuracy'])

## 4.3. Training

In [ ]:
model_checkpoint = ModelCheckpoint(save_model_name, monitor='val_acc', mode = 'max', save_best_only=True, verbose=1)
    
history = model.fit_generator(
                    generator=batch_generator(x_train, y_train, weight, batch_size),
                    steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=batch_generator(x_valid, y_valid, 1.0, 1),
                    validation_steps=x_valid.shape[0],
                    epochs=epochs,
                    callbacks=[model_checkpoint], 
                    shuffle=True,
                    verbose=1)

model.save(save_end_name)

In [ ]:
model = load_model(transfar_model_01)

# 5. Create prediction

In [ ]:
def predict_result(model,x_test,img_size_target):
    preds_test  = 0.250 * model.predict(x_test); print("Done - preds_test: 0, 1")
    preds_test += 0.125 * model.predict(do_brightness_aug(x_test, 8)); print("Done - preds_test: 2")
    preds_test += 0.125 * model.predict(do_brightness_aug(x_test, -8)); print("Done - preds_test: 3")
    
    x_test4 = do_flip_transpose_aug(x_test, 4)
    preds_test4 = model.predict(x_test4)
    preds_test4 = preds_test4
    preds_test += 0.250 * preds_test4; print("Done - preds_test: 4, 5")
    preds_test4 = model.predict(do_brightness_aug(x_test4, 8))
    preds_test4 = preds_test4
    preds_test += 0.125 * preds_test4; print("Done - preds_test: 6")
    preds_test4 = model.predict(do_brightness_aug(x_test4, -8))
    preds_test4 = preds_test4
    preds_test += 0.125 * preds_test4; print("Done - preds_test: 7")
    del x_test4, preds_test4; gc.collect()
    
    return preds_test

In [ ]:
preds_valid = predict_result(model, x_valid, img_size_target)

In [ ]:
from sklearn.metrics import accuracy_score
thresholds = np.linspace(0.2, 0.8, 101)
accs = np.array([accuracy_score(y_valid, preds_valid > threshold) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(accs) 
acc_best = accs[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

plt.plot(thresholds, accs)
plt.plot(threshold_best, acc_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, acc_best))
plt.legend()
plt.grid(True)

In [ ]:
preds_valid_bi = np.squeeze(preds_valid > threshold_best)*1

# 6. Now, it's time to train ourselves!

## 6.1. Preparation

In [ ]:
x_train_org = np.array([downsample(np.squeeze(x_train_org[i])) for i in np.arange(x_train_org.shape[0])])
y_train_org = np.array([downsample(np.squeeze(y_train_org[i])) for i in np.arange(y_train_org.shape[0])])
x_valid_org = np.array([downsample(np.squeeze(x_valid_org[i])) for i in np.arange(x_valid_org.shape[0])])
y_valid_org = np.array([downsample(np.squeeze(y_valid_org[i])) for i in np.arange(y_valid_org.shape[0])])

In [ ]:
import cv2
from IPython.display import display, Image
def cvshow(image, format='.png', rate=255 ):
    decoded_bytes = cv2.imencode(format, image*rate)[1].tobytes()
    display(Image(data=decoded_bytes))
    return

## 6.2. Without salt

In [ ]:
def imgtile(imgs,tile_w):
    assert imgs.shape[0]%tile_w==0,"'imgs' cannot divide by 'th'."
    r=imgs.reshape((-1,tile_w)+imgs.shape[1:])
    return np.hstack(np.hstack(r))

print("Without salt")
tiled = imgtile(x_train_org[y_train==0][:20],10)
cvshow(tiled)

## 6.3. With salt

In [ ]:
def imgtile(imgs,tile_w):
    assert imgs.shape[0]%tile_w==0,"'imgs' cannot divide by 'th'."
    r=imgs.reshape((-1,tile_w)+imgs.shape[1:])
    return np.hstack(np.hstack(r))

print("With salt")
tiled = imgtile(x_train_org[y_train==1][:20],10)
cvshow(tiled)

## 6.4. My trained model does not know the infomation of salt area, but...

In [ ]:
def imgtile(imgs,tile_w):
    assert imgs.shape[0]%tile_w==0,"'imgs' cannot divide by 'th'."
    r=imgs.reshape((-1,tile_w)+imgs.shape[1:])
    return np.hstack(np.hstack(r))

tiled = imgtile(x_train_org[y_train==1][:20],10)
cvshow(tiled)
tiled = imgtile(y_train_org[y_train==1][:20],10)
cvshow(tiled)

# 7. Please beat my model!

## Q. Which images contain salt?

In [ ]:
start = np.random.randint(x_valid_org.shape[0])

tiled = imgtile(x_valid_org[start:start+5],5)
cvshow(tiled)

## What's your answer??

## My model's answer is...

In [ ]:
preds_valid_bi[start:start+5].tolist()

## A. Answer

In [ ]:
answer = np.any(y_valid_org[start:start+5]==1, axis=(1,2))*1
print("{}".format(answer.tolist()))
tiled = imgtile(x_valid_org[start:start+5],5)
cvshow(tiled)
tiled = imgtile(y_valid_org[start:start+5],5)
cvshow(tiled)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

fpr, tpr, thresholds = metrics.roc_curve(y_valid, preds_valid)

auc = metrics.auc(fpr, tpr)

plt.figure(figsize=(8,8))
plt.rcParams["font.size"] = 18
plt.plot(fpr, tpr, label='ROC curve (area = %.3f)'%auc)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('ROC curve (acc = %.3f)'%accuracy_score(y_valid, preds_valid_bi))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

# 8. Model structure

In [ ]:
from keras.utils import plot_model
from keras.preprocessing.image import load_img
plot_model(model, show_shapes=True, show_layer_names=False, to_file='./model.png')
im = load_img('./model.png')
im

In [ ]:
model.summary()

In [ ]:
import time
from datetime import datetime, timedelta, timezone
from contextlib import contextmanager
JST = timezone(timedelta(hours=+9), 'JST')
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} -> done in {:.0f}s".format(title, time.time() - t0))
    print("Executed time -> {}\n".format(datetime.now(JST)))

with timer('end'):
    print('finish!')